<a href="https://colab.research.google.com/github/nattycep/FIL_Geosensing/blob/main/DataVis_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Import, Visualization, and Troubleshooting with Python

**Expected study time :** 45 min (depending on your background)

**Learning outcomes :**

*Adapt an existing Python script in google colab in order to:*
- Load data from your geosensor
- Import data into Python
- Visualize the data
- Refine and export figures for presentation
- Perform basic statistics to support a scientific finding 

**External References**
- the Scipy.stat module https://docs.scipy.org/doc/scipy/reference/stats.html
- Matplotlib cheat sheet https://github.com/matplotlib/cheatsheets



# 1.   Load Packages



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats

# 2.   Load Data 


Make sure that you data makes sense. 
Check that the dates imported correctly. 
Look at the header and variable names. 

If you import from github, make sure that you are using the "raw" prefix after the https://   ... 
On google Colab, you also can upload local files.  To do that, click on the "file" icon on the left side and then the file with an arrow on it. 




# 3.   Make Figures: Preliminary, Refine, Export

Ideas 

3.1   Time Series


```
fig, ax = plt.subplots()
ax.plot(X,Y)
fig.show()

```



3.2   Scatter of one variable against another


```
ax.scatter(X, Y)
```



3.3   Boxplot of one treatment against another
Fist need to reshape data so that it is is in columns by groups. 



```
ax.boxplot(Z)
```




3.4   Comparison with a model

```

Ym = function(X)
ax.plot(X, Y, X, Ym)
```

3.5   Convert data resolution to hourly, daily, etc. 
3.6   Change analysis
3.7   Heatmap
```
ax.imshow(Z)
```

3.8   Group scatter  


3.X   Save your figure



```
fig.savefig("mydatavis.png", dpi=300)

fig.savefig("mydavavis.pdf")
```






# 4.  Perform Statistical Tests 
